In [5]:
import pandas as pd
import numpy as np
import openrouteservice as ors
from dotenv import load_dotenv
from pyonemap import OneMap
import os
import requests
import json
import time
from collections import namedtuple
import connectivity_functions as cf
from datetime import datetime,timedelta

In [6]:
load_dotenv()

False

In [7]:
directory = os.getcwd()

os.chdir(directory)

hdbCentroids_df = pd.read_csv(r"..\data\hdb_cluster_centroids.csv",header = None,names = ['Latitude','Longitude'])
mrt_stations_df = pd.read_csv(r"..\data\mrt_station_final.csv",usecols = [1,2,3])

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\hdb_cluster_centroids.csv'

In [11]:
hdbCentroids_df.info()
mrt_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   550 non-null    float64
 1   Longitude  550 non-null    float64
dtypes: float64(2)
memory usage: 8.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MRT.Name   175 non-null    object 
 1   Latitude   175 non-null    float64
 2   Longitude  175 non-null    float64
dtypes: float64(2), object(1)
memory usage: 4.2+ KB


In [12]:
mrt_stations_df.sort_values(by='MRT.Name', inplace=True)

mrt_stations_df

,MRT.Name,Latitude,Longitude
12,ADMIRALTY MRT STATION,1.440589,103.800990
9,ALJUNIED MRT STATION,1.316433,103.882906
52,ANG MO KIO MRT STATION,1.369429,103.849455
115,BAKAU LRT STATION,1.387994,103.905415
54,BANGKIT LRT STATION,1.380022,103.772647
...,...,...,...
158,WOODLANDS SOUTH MRT STATION,1.427488,103.792730
80,WOODLEIGH MRT STATION,1.339190,103.870818
77,YEW TEE MRT STATION,1.397298,103.747358
79,YIO CHU KANG MRT STATION,1.381499,103.845171


In [13]:
#create a psuedo index for my residential centroids df
hdbCentroids_df['index'] = hdbCentroids_df.index

hdbCentroids_df['index']

#create a psuedo index for my residential centroids df
mrt_stations_df['index'] = mrt_stations_df.index

mrt_stations_df['index']

##create a dummy variable to cross join on 
hdbCentroids_df['join_key'] = "A"
mrt_stations_df['join_key'] = "A"

#Cross join to obtain combinations of all possible pairings between MRTs and Residential Centroids
combined_df = pd.merge(hdbCentroids_df, mrt_stations_df, on='join_key')

print(combined_df)

       Latitude_x  Longitude_x  index_x join_key                     MRT.Name  \
0        1.282627   103.826023        0        A        ADMIRALTY MRT STATION   
1        1.282627   103.826023        0        A         ALJUNIED MRT STATION   
2        1.282627   103.826023        0        A       ANG MO KIO MRT STATION   
3        1.282627   103.826023        0        A            BAKAU LRT STATION   
4        1.282627   103.826023        0        A          BANGKIT LRT STATION   
...           ...          ...      ...      ...                          ...   
96245    1.271741   103.823949      549        A  WOODLANDS SOUTH MRT STATION   
96246    1.271741   103.823949      549        A        WOODLEIGH MRT STATION   
96247    1.271741   103.823949      549        A          YEW TEE MRT STATION   
96248    1.271741   103.823949      549        A     YIO CHU KANG MRT STATION   
96249    1.271741   103.823949      549        A           YISHUN MRT STATION   

       Latitude_y  Longitud

In [14]:
#Defining a function that calculates the Euclidean Distance between two points using Haversine Method?
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    distance = R * c
    return distance

In [15]:
coordinate_pair = namedtuple('coordinate_pair',['lat_x','lon_x','lat_y','lon_y'])

In [16]:
#Apply function to dataframe and store distances in new column 'euclidean distance'
combined_df['euclidean_distance'] = haversine(combined_df['Latitude_x'], combined_df['Longitude_x'], combined_df['Latitude_y'], combined_df['Longitude_y'])

#Group by residential centroid, filter out the closest MRT by distance for each centroid into another dataframe and reset its index
result_df = combined_df.loc[combined_df.groupby('index_x')['euclidean_distance'].idxmin()].reset_index()

result_df

#Create a new column 'coordinate_pair' to store coordinate pairs to pass to openrouteservice API direction query
# result_df['coordinate_pair'] = list(zip(result_df['Longitude_x'], result_df['Latitude_x'], result_df['Longitude_y'], result_df['Latitude_y']))
# result_df['coordinate_pair'] = result_df['coordinate_pair'].apply(lambda x: [[x[0], x[1]], [x[2], x[3]]])
result_df['coordinate_pair'] = result_df.apply(lambda x: coordinate_pair(x['Latitude_x'], x['Longitude_x'], x['Latitude_y'], x['Longitude_y']), axis=1)

#create an empty column 'route' to later store query response
result_df['cycle_route'] = np.nan
result_df['bus_route'] = np.nan

In [114]:
one_map_email = os.getenv("ONE_MAP_EMAIL")
one_map_password = os.getenv("ONE_MAP_PASSWORD")
payload = {
        "email": one_map_email,
        "password": one_map_password
      }
api_key = requests.request("POST", "https://www.onemap.gov.sg/api/auth/post/getToken", json=payload)
api_key = api_key.json()["access_token"]

In [115]:
onemap = OneMap(api_key)

In [43]:
def get_cycle_route(coordinate_pair):
    time.sleep(0.5)
    try:
        return onemap.routing.route(start_lat=coordinate_pair.lat_x, 
                                    start_lon=coordinate_pair.lon_x, 
                                    end_lat=coordinate_pair.lat_y, 
                                    end_lon=coordinate_pair.lon_y, 
                                    routeType="cycle")
    except Exception as e:
        print(f"Error: {e}")
        return None
    
    # def get_route(coordinate_pair):
#     time.sleep(2)
#     try:
#         return client.directions(coordinate_pair, profile='cycling-regular', format='geojson', validate=False)
#     except Exception as e:
#         print(f"Error: {e}")
#         return None

In [219]:
def get_bus_route(coordinate_pair):
    # Get today's date
    today = datetime.now()
    # Calculate yesterday's date
    yesterday = today - timedelta(days=1)
    time.sleep(0.5)
    try:
        return onemap.routing.route(start_lat=coordinate_pair.lat_x, 
                                    start_lon=coordinate_pair.lon_x, 
                                    end_lat=coordinate_pair.lat_y, 
                                    end_lon=coordinate_pair.lon_y, 
                                    routeType="pt",
                                    mode = "BUS",
                                    date =  yesterday.strftime('%m-%d-%Y'),
                                    time = '18:00:00',
                                    maxWalkDistance = 2000)
    except Exception as e:
        print(f"Error: {e}")
        return None

In [23]:
result_df['cycle_route'] = result_df['coordinate_pair'].apply(get_cycle_route)

In [220]:
#Not working


Error: 404 Client Error: Not Found for url: https://www.onemap.gov.sg/api/public/routingsvc/route?start=1.28262693083003%2C103.8260229912674&end=1.286102536%2C103.8274451&routeType=pt&date=03-27-2024&time=18%3A00%3A00&mode=BUS&maxWalkDistance=2000&numItineraries=3


KeyboardInterrupt: 

In [182]:
def get_distance(route):
    try:
        return route['route_summary']['total_distance']/1000 #convert m to km
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None

result_df['distance'] = result_df['cycle_route'].apply(get_distance)

def get_time(route):
    try:
        return route['route_summary']['total_time']/60 #convert second to minutes
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None
    

result_df['cycle_duration'] = result_df['cycle_route'].apply(get_time)


def get_bus_time(route):
    try: 
        return route['plan']['itineraries'][0]['duration']/60 # convert seconds to minutes
    except (KeyError, IndexError,TypeError) as e:
        print(f"Error: {e}")
        return None

result_df['bus_duration'] = result_df['cycle_route'].apply(get_bus_time)

# def get_distance(route):
#     try:
#         return route['features'][0]['properties']['segments'][0]['distance']
#     except (KeyError, IndexError,TypeError) as e:
#         print(f"Error: {e}")
#         return None

# result_df['distance'] = result_df['route'].apply(get_distance)

# def get_time(route):
#     try:
#         return route['features'][0]['properties']['segments'][0]['duration']
#     except (KeyError, IndexError,TypeError) as e:
#         print(f"Error: {e}")
#         return None
    

# result_df['duration'] = result_df['route'].apply(get_time)

Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error: 'plan'
Error:

In [183]:
result_df['time_difference'] =  result_df['cycle_duration'] - result_df['bus_duration']

In [184]:
result_df

,index,Latitude_x,Longitude_x,index_x,join_key,MRT.Name,Latitude_y,Longitude_y,index_y,euclidean_distance,coordinate_pair,cycle_route,bus_route,distance,cycle_duration,bus_duration,time_difference
0,159,1.282627,103.826023,0,A,TIONG BAHRU MRT STATION,1.286103,103.827445,87,0.417554,"(1.28262693083003, 103.8260229912674, 1.286102...",{'status_message': 'Found route between points...,None,0.460,3.016667,None,NaN
1,336,1.380156,103.879420,1,A,TONGKANG LRT STATION,1.389348,103.885844,84,1.246891,"(1.3801559906311147, 103.87941955476325, 1.389...",{'status_message': 'Found route between points...,None,2.648,16.500000,None,NaN
2,425,1.380869,103.748413,2,A,KEAT HONG LRT STATION,1.378603,103.749056,125,0.261871,"(1.380868921822081, 103.74841283436977, 1.3786...",{'status_message': 'Found route between points...,None,0.621,4.416667,None,NaN
3,691,1.344200,103.959075,3,A,UPPER CHANGI MRT STATION,1.341740,103.961473,82,0.381959,"(1.3442002658092747, 103.95907516602809, 1.341...",{'status_message': 'Found route between points...,None,0.955,6.200000,None,NaN
4,700,1.438353,103.798333,4,A,ADMIRALTY MRT STATION,1.440589,103.800990,12,0.386099,"(1.4383527950723751, 103.79833297097744, 1.440...",{'status_message': 'Found route between points...,None,0.537,3.633333,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,95459,1.349409,103.719539,545,A,LAKESIDE MRT STATION,1.344259,103.720949,93,0.593763,"(1.3494094610862557, 103.71953880510551, 1.344...",{'status_message': 'Found route between points...,None,0.736,5.150000,None,NaN
546,95616,1.376455,103.888071,546,A,HOUGANG MRT STATION,1.371292,103.892381,7,0.747724,"(1.376454956138211, 103.888070568638, 1.371292...",{'status_message': 'Found route between points...,None,1.572,10.016667,None,NaN
547,95811,1.390286,103.880154,547,A,LAYAR LRT STATION,1.392080,103.880030,18,0.199922,"(1.3902862288885647, 103.88015441726768, 1.392...",{'status_message': 'Found route between points...,None,0.444,3.283333,None,NaN
548,95902,1.362677,103.854576,548,A,ANG MO KIO MRT STATION,1.369429,103.849455,52,0.942129,"(1.3626769432417178, 103.8545755938334, 1.3694...",{'status_message': 'Found route between points...,None,1.557,10.550000,None,NaN


In [216]:
def get_centroid_name(row):
    geocode = onemap.reverseGeocode.revGeoCode(row['Latitude_x'], row['Longitude_x'])
    if geocode['GeocodeInfo'][0]['BUILDINGNAME'] != "NIL" and geocode['GeocodeInfo'][0]['BUILDINGNAME'] != "MULTI STOREY CAR PARK":
            return geocode['GeocodeInfo'][0]['BUILDINGNAME']
    else:
        if geocode['GeocodeInfo'][0]['BLOCK'] != "NIL":
            return geocode['GeocodeInfo'][0]['BLOCK'] + " " + geocode['GeocodeInfo'][0]['ROAD']
        else:
            return geocode['GeocodeInfo'][0]['ROAD']

In [217]:
result_df['centroid_name'] = result_df.apply(get_centroid_name, axis=1)

In [198]:
ors_key = os.getenv("ORS_API_KEY")

client = ors.Client(key= ors_key)

In [201]:
result_df['suitability'] = result_df['cycle_route'].apply(lambda x: cf.get_path_suitability(client,x))

C:\Users\leoqi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\leoqi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\leoqi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\leoqi\AppData\Lo

In [ ]:
result_df['steepness'] = result_df['cycle_route'].apply(lambda x: cf.get_path_steepness(client,x))

In [218]:
new_result_df = result_df. loc[:, result_df. columns != 'route']
#new_result_df.to_csv(r"..\data\HDB_Centroid_MRT pairing data.csv")
new_result_df.to_csv(r"C:\Users\leoqi\Downloads\HDB_Centroid_MRT pairing data.csv")
routes = result_df['cycle_route'].copy(deep = True)
#routes.to_json(r'..\data\HDB_MRT_cycle_routes.json', orient='records')
routes.to_json(r'C:\Users\leoqi\Downloads\HDB_MRT_cycle_routes.json', orient='records')

In [39]:
result_df[result_df['route'].isna()]

,index,Latitude_x,Longitude_x,index_x,join_key,MRT.Name,Latitude_y,Longitude_y,index_y,euclidean_distance,coordinate_pair,route,distance,duration,centroid_name


In [40]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               275 non-null    int64  
 1   Latitude_x          275 non-null    float64
 2   Longitude_x         275 non-null    float64
 3   index_x             275 non-null    int64  
 4   join_key            275 non-null    object 
 5   MRT.Name            275 non-null    object 
 6   Latitude_y          275 non-null    float64
 7   Longitude_y         275 non-null    float64
 8   index_y             275 non-null    int64  
 9   euclidean_distance  275 non-null    float64
 10  coordinate_pair     275 non-null    object 
 11  route               275 non-null    object 
 12  distance            275 non-null    float64
 13  duration            275 non-null    float64
 14  centroid_name       275 non-null    object 
dtypes: float64(7), int64(3), object(5)
memory usage: 32.4+ KB

In [51]:
hdb_df = pd.read_csv(r"../data/HDB_Centroid_MRT pairing data.csv")

In [52]:
import requests
import json

def fetch_itinerary(from_lat, from_lon, to_lat, to_lon):
    # Define the GraphQL endpoint
    endpoint = 'http://localhost:8080/otp/gtfs/v1'
    
    # Construct the GraphQL query
    query = """
    query EgQuery($time: String, $date: String, $from: InputCoordinates, $to: InputCoordinates, $numItineraries: Int, $walkReluctance: Float) {
      plan(
        time: $time
        date: $date
        from: $from
        to: $to
        numItineraries: $numItineraries
        walkReluctance: $walkReluctance
      ) {
        itineraries {
          legs {
            mode
            from {
              name
              vertexType
            }
            to {
              name
              vertexType
            }
          }
          duration
        }
      }
    }
    """

    # Define the variables for your query
    variables = {
        'time': '18:00:00',
        'date': '2023-03-01',
        'from': {'lat': from_lat, 'lon': from_lon},
        'to': {'lat': to_lat, 'lon': to_lon},
        'numItineraries': 1,
        'walkReluctance': 4.0
    }

    # Set up the headers
    headers = {
        'Content-Type': 'application/json'
    }

    # Send the request to the GraphQL API
    response = requests.post(endpoint, headers=headers, json={'query': query, 'variables': variables})

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"GraphQL query failed to run with a {response.status_code}")



In [53]:
def apply_fetch_itinerary(row):
    try:
        response = fetch_itinerary(
            from_lat=row['Latitude_x'], 
            from_lon=row['Longitude_x'], 
            to_lat=row['Latitude_y'], 
            to_lon=row['Longitude_y']
        )
        bus_route = []
        # Extract relevant information from response
        if response and 'data' in response and 'plan' in response['data'] and 'itineraries' in response['data']['plan']:
            itinerary = response['data']['plan']['itineraries'][0]  # Taking the first itinerary
            duration = itinerary['duration'] if 'duration' in itinerary else None
            for leg in itinerary['legs']:
                if leg['mode'] == 'BUS':
                    # Append bus route information for each bus leg
                    from_stop = leg['from']['name']
                    to_stop = leg['to']['name']
                    bus_route.append(f"{from_stop} to {to_stop}")
            # Further extraction can be done based on the structure of your response
            return bus_route, duration/60  # Convert seconds to minutes
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching itinerary: {e}")
        return None, None

# Apply the function to each row of the DataFrame
hdb_df[['bus_route', 'bus_duration']] = hdb_df.apply(lambda row: pd.Series(apply_fetch_itinerary(row)), axis=1)
hdb_df['time_difference'] = hdb_df['cycle_duration'] - hdb_df['bus_duration']
hdb_df.to_csv(r"../data/HDB_Centroid_MRT pairing data.csv")